## 统计电生理信号中，接收到打标信号的时间信息

In [ ]:
# 接下来是EphysProcessor的使用示例，其目的是为了获取电生理数据的打标时间序列，包括以秒为单位的时间戳和对应的索引。
# 这里假设你已经安装了kiana库，并且有一个名为EphysProcessor的类可以处理电生理数据。
import numpy as np
from kiana import EphysProcessor
ephys_processor = EphysProcessor(session_id='20250321', 
                             root_path="~/nas/+largescale/time_analysis/HPC_time_analysis",
                             f_s=30000,)
ephys_processor.load_and_merge_data() # Ephys的数据是一个json文件，包含了多个控制器的电生理数据
# ephys_processor.filter_by_controller(drop=['B6']) #你可以使用filter_by_controller来过滤掉不需要的控制器
# ephys_processor.filter_by_time(start_time_str='2025-03-21 00:00:00',
#                                end_time_str='2025-03-22 00:00:00') # 你也可以用filter_by_time来过滤掉不需要的时间段
ephys_processor.process_controllers() #正式的处理过程

# 通过get_result来获取结果，结果是一个字典，包含times和indices. 
# 这个地方是展示用的，后续
# A1_series = ephys_processor.get_result('A1')
# ephys_event_time_array = np.array(A1_series['times'])
# ephys_event_indices_array = np.array(A1_series['indices'])

## 计算Monkeylogic的记录文件中，发出打标信号的时间信息

In [ ]:
import json
import numpy as np
from pathlib import Path
import pandas as pd
from kiana import BehavioralProcessor,MatLoader,DataFrameLoader,SeqLoader

# 准备Monkeylogic数据的路径
MAT_ROOTFILE = Path("~/nas/jsy/sync/Data/monkeylogic").expanduser()
mat_file = next(MAT_ROOTFILE.glob("8-v15/250321_162010_2017049_condition.mat"))
# 这里的notation_dict是一个字典，包含了不同的行为代码和对应的时间段，可以不写
notation_dict = {"MDD multi-coherence": (1, 549), 
                 "MDD delaytime=1200ms": (550, 692), 
                 "MDD delaytime=2100ms": (693, 851), 
                 "MDD delaytime=3000ms": (852, 1008),} 

# 读取动捕数据，此处用于展示DataFrameLoader的使用
TASK_TIME_PATH = Path("~/nas/zch/motion_trigger").expanduser()
task_time_file = next(TASK_TIME_PATH.glob("*.json"))
with task_time_file.open("r") as f:
    data = json.load(f)
start_time_list = [item['start'] for item in data]
end_time_list = [item['end'] for item in data]
length_list = [item['length'] for item in data]
end_time_array = np.array([float(x) for x in end_time_list], dtype=np.float64)
# 对于DataFrameLoader来说，它只要要求是一个DataFrame，里面包含一个名为'EventTime'的列即可,
# 其他列可以当作备注，或者不需要。
motion_data = pd.DataFrame({'EventTime': end_time_array, })

# 处理行为数据
# 创建BehavioralProcessor实例
behav_processor = BehavioralProcessor()

# # 可以用session recipe来处理数据
# session_recipe = [
#     {'segment_name': 'mat_file', 
#      'loader': MatLoader(notation_map=notation_dict), 
#      'source': mat_file, 
#      'anchor_query': "BehavioralCode == 19",
#       'slice_by': {'TrialID': (850, 900)} # 筛选 TrialID 从 850 到 900
#         },
#     {'segment_name': 'Motion', 
#      'loader': DataFrameLoader(), 
#      'source': motion_data,},
# ]
# behav_processor.build_from_recipe(session_recipe)

# # 或者，可以使用一种更加自然的方式来处理session的segment
behav_processor.add_segment(segment_name='mat_file',
                    loader=MatLoader(notation_map=notation_dict), # 每个文件可以有一个自己的notation_dict
                    source=mat_file).with_anchors("BehavioralCode == 19") # 可以指定打标点，这里的语句是pandas的查询语句
                    # .with_slicing({'TrialID': (850, 900)}) # 或者你可以对同一个mat进行分段，使用with_slicing来指定切片范围
behav_processor.add_segment(segment_name='Motion',
                    loader=DataFrameLoader(), #如果你的DataFrame中有trial ID信息，可以选择用trial_id_col参数传递给DataFrameLoader
                    source=motion_data) # 同样的，这里也可以用.with_anchors或者.with_slicing来指定打标点或者切片范围，
                                        # 如果不写，默认为全是打标点
# behav_processor.add_segment(segment_name='MotionCapture',
#                     loader=TrcLoader(pure=True), #如果你的DataFrame中有trial ID信息，可以选择用trial_id_col参数传递给DataFrameLoader
#                     source="/path/to/data.trc") # 同样的，这里也可以用.with_anchors或者.with_slicing来指定打标点或者切片范围，
#                                         # 如果不写，默认为全是打标点
# behav_processor.add_segment(segment_name='Seq',
#                     loader=SeqLoader(), #如果你的DataFrame中有trial ID信息，可以选择用trial_id_col参数传递给DataFrameLoader
#                     source="/path/to/data.seq").with_kwargs(Timezone='Asia/Shanghai') # 同样的，这里也可以用.with_anchors或者.with_slicing来指定打标点或者切片范围，
#                                         # 如果不写，默认为全是打标点
behav_processor.build() # 用build来完成行为数据的处理

# 使用add_sync_context来完成电生理数据与行为数据的对齐，并添加至dataframe中
# controller_list = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 
#               'B1', 'B2', 'B3', 'B4', 'B5', 'B6']
controller_list = ['A1']
for controller_name in controller_list:
    series = ephys_processor.get_result(controller_name) # 获取电生理数据的时间序列和索引，或者自己构建
    # 使用add_sync_context来添加同步上下文，采样率是为了推算以秒为单位的时间
    # ephys_times, ephys_indices, 和sampling_rate三者中给两个即可，将通过这两个推算出剩下的那一个。
    # 这里的ephys_times是以秒为单位的时间戳，ephys_indices是对应的采样点索引。
    # 考虑到不同采集系统的采样率轻微浮动，这里建议提供indices，因为它最精确。
    behav_processor.add_sync_context(controller_name, 
                                    #  ephys_times=series['times'], 
                                     ephys_indices= series['indices'], 
                                     sampling_rate=30000)

# 最后，用get_final_dataframe来获取最终的事件数据框
event_dataframe = behav_processor.get_final_dataframe()
event_dataframe # 可以预览结果

# 结果中，对于每个匹配的controller, 都会增加两列，
# 一列是EphysTime_{controller_name}，表示电生理数据的时间（以秒为单位），
# 一列是EphysIndice_{controller_name}，表示电生理数据的索引（以采样点为单位）。
# 另外，还会反向计算出AbsoluteDateTime列，作为人类可读的时间参考。
# 由于电脑系统时间的误差，这个参考不一定准确，因此切勿将它作为后续分析的依据。
# 能作为依据的只有EphysTime和EphysIndice，而其中最准确的是EphysIndice.

In [ ]:
# 从event_dataframe中提取出需要的行
query_string = "Notation == 'MDD multi-coherence' and TrialError == 0" # 如果前面定义了字典,这里的notation就很方便
# 你可以根据需要修改query_string来筛选不同的事件
# 例如，如果你想筛选BehavioralCode为30, 31, 42 可以使用以下查询语句：
# query_string = "BehavioralCode in [30, 31, 42] and Notation == 'MDD multi-coherence' and TrialError == 0"
event_dataframe_filtered = event_dataframe.query(query_string)
event_dataframe_filtered # 预览结果

## 读取 Kilosort 的运行结果数据，主要是 spike time 和对应的 cluster id

In [ ]:
from pathlib import Path
F_S = 30000
DATA_DIR =  Path("~/nas/+largescale/kilosort_result/preprocessed").expanduser()
DAILY_DIR = next(DATA_DIR.glob("20250321*"))
PROBE_NAME = "1"
probe_dir = DAILY_DIR / PROBE_NAME  # 选择Probe 1的目录
# 验证probe_dir / kilosort4 是否存在
if not (probe_dir / 'kilosort4').exists():
    raise FileNotFoundError(f"Probe directory {probe_dir / 'kilosort4'} does not exist.")

probe_ks_result = probe_dir / "kilosort4" # No need to check if it exists, since probe_dir uses kilosort4's parent folder
st_file = str(probe_ks_result / "spike_times.npy")
sc_file = str(probe_ks_result / "spike_clusters.npy")

st_data = np.load(st_file).squeeze()
clu_data = np.load(sc_file).squeeze()
st = st_data[st_data >= 0]
clu = clu_data[st_data >= 0]

clu_info = pd.read_csv(probe_ks_result / "cluster_info.tsv", sep="\t")
clu_info = clu_info.set_index('cluster_id') # 设置索引，以便提取cluster的channel

## 绘制某个cluster的Raster plot和PSTH图

In [ ]:
# 这不是一个必要的步骤，但是如果希望结果更清晰（相同条件的trail绘制到一起），可以对event_dataframe_filtered进行排序
# 这里假设你只关心'MDD multi-coherence'的事件，并且希望筛选出TrialError为0的事件
event_dataframe_filtered = event_dataframe.query("Notation == 'MDD multi-coherence' and TrialError == [0,6]")
# 对筛选后的数据进行排序，此处演示用coherence进行排序，这是把相同coherence的trial放到一起，以便后续绘制
event_dataframe_filtered.sort_values(by=['TrialError', 'Coherence', 'EventTime'], inplace=True)
# 重置索引，以便后续处理
event_dataframe_filtered.reset_index(drop=True, inplace=True)
# event_dataframe_filtered # 预览结果

In [ ]:
PROBE_CONTROLLER_PROBE = { "1": "A1", "2": "A1", "3": "A2", "4": "A2", "5": "A3", "6": "A3", 
                          "11R": "A4", "7": "A4", "12R": "A5", "8": "A5", "13R": "A6", "10R": "A6",
                           "9": "B1", "16R": "B1", "15": "B2", "14": "B3", "13": "B3", "16": "B4",
                           "12": "B4", "15R": "B5", "11": "B5", "14R": "B6", "10": "B6"}
controller_name = PROBE_CONTROLLER_PROBE[PROBE_NAME]

# 19 21 22 23 24 25 26 30 31 42
# 使用query语句，读取不同BehavioralCode对应的事件时间列表，这里读取的是A1controller的事件时间
event_19_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 19")[f"EphysTime_{controller_name}"].to_list())
event_21_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 21")[f"EphysTime_{controller_name}"].to_list())
event_22_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 22")[f"EphysTime_{controller_name}"].to_list())
event_23_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 23")[f"EphysTime_{controller_name}"].to_list())
event_24_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 24")[f"EphysTime_{controller_name}"].to_list())
event_25_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 25")[f"EphysTime_{controller_name}"].to_list())
event_26_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 26")[f"EphysTime_{controller_name}"].to_list())
event_30_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 30")[f"EphysTime_{controller_name}"].to_list())
event_31_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 31")[f"EphysTime_{controller_name}"].to_list())
event_42_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 42")[f"EphysTime_{controller_name}"].to_list())
event_41_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 41")[f"EphysTime_{controller_name}"].to_list())
event_40_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 40")[f"EphysTime_{controller_name}"].to_list())
event_touchstart_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 'TouchStart'")[f"EphysTime_{controller_name}"].to_list())
event_touchend_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 'TouchEnd'")[f"EphysTime_{controller_name}"].to_list())
event_joystickstart_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 'Button1Start'")[f"EphysTime_{controller_name}"].to_list())
event_joystickend_time_list = np.array(event_dataframe_filtered.query("BehavioralCode == 'Button1End'")[f"EphysTime_{controller_name}"].to_list())

# 构建每个trial的condition列表，使用BehavioralCode为19主要是因为每个trial都只有一个BehavioralCode为19的事件，用其他方法构建也行
# 这个trial_conditions是一个列表，包含了每个trial的条件信息
# 将作为后续按情况绘制psth和分组raster plot y轴的依据
trial_conditions = [f"Coherence: {coherence}%" for coherence in 
                    event_dataframe_filtered.query("BehavioralCode == 19").Coherence.to_list()]
# 如果按direction分组，那么就可以使用以下方式来构建trial_conditions
# trial_conditions = [f"Direction: {direction}" for direction in 
#                     event_dataframe_filtered.query("BehavioralCode == 19").Direction.to_list()]

# 为每个时间列表附上对应的人类可读的事件信息，将用于图例绘制
extra_events = {
    "猴子拉上杆": event_21_time_list,
    "猴子松开杆": event_joystickend_time_list,
    # "猴子拉杆已满joystick_warming_time时间": event_22_time_list,
    "注视点出现": event_23_time_list,
    "注视点消失": event_24_time_list,
    "白点群出现": event_25_time_list,
    "白点群消失": event_26_time_list,
    # "白点群消失满delay_timing(2)时间": event_30_time_list, #你可以任意注销掉一些事件，这样它们就不会在图例中出现。
    "选择点出现": event_31_time_list,
    "猴子摸屏幕": event_touchstart_time_list,
    "猴子松开屏幕": event_touchend_time_list,
    "猴子做出正确选择": event_42_time_list,
    "给水": event_40_time_list,
    "屏幕变黑":event_41_time_list
}

In [ ]:
from kiana import SpikeTrainAnalyzer
from matplotlib import pyplot as plt
import matplotlib.font_manager as fm
import os

# 选择需要分析的cluster ID，这里假设你已经知道了哪些cluster是显著的
significant_clusters = [26, 32, 110, 175, 184, 190, 205, 208, 234, 275, 519, 586, 599, 609, 667, 697, 720, 728, 872, 877, 901]

for clu_id in significant_clusters[:1]:
    # 从kilosort结果的st和clu中提取出对应cluster的spike train，注意需要以秒为单位，因此除以采样率F_S
    spike_train = np.array(st[clu==clu_id]) / F_S

    font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
    my_font = fm.FontProperties(fname=font_path) if os.path.exists(font_path) else None

    # 构建事件窗口数组，这是一个2维数组，每行代表了一个trial的时间窗口的开始和结束时间
    # 这里假设每个trial的事件窗口是以事件26的时间为中心，前后各5秒
    # 你可以根据需要调整这个时间窗口的大小和中心
    event_start_end_array = np.column_stack((event_26_time_list-5, event_26_time_list+8))

    # 创建SpikeTrainAnalyzer实例
    analyzer = SpikeTrainAnalyzer(
        spike_train=spike_train, 
        event_windows=event_start_end_array, # <--- 事件窗口数组
        alignment_times=event_26_time_list, # <--- 对齐时间列表，代表你希望对齐的事件时间，之后的spike train会以此为基准
        extra_events=extra_events, # <--- 附加事件列表，用于绘制其他事件和图例
        font_prop=my_font  # <--- 字体属性，如果需要中文支持，可以设置为Noto Sans CJK等支持中文的字体
    )
    # 计算PSTH，使用高斯平滑方法，标准差为50毫秒，分辨率为1毫秒，分析窗口为[-5, 5]秒
    analyzer.calculate_rates(
        mode='gaussian', 
        gaussian_std=40e-3,
        high_res_bin=1e-3,
        analysis_window=[-5,5] # 分析窗口为[-5, 5]秒，相对于对齐事件而言。或者你可以设置更小的窗口
    )
    # 你也可以使用binned方法，指定bin大小为10毫秒
    # analyzer.calculate_rates(mode='binned', bin_size=10e-3)

    # 绘制PSTH图，输入trial_labels可以按condition绘制不同条件的psth图
    # analyzer.plot_psth(title=f"Cluster {clu_id} PSTH Analysis, on channel {clu_info.at[clu_id, 'ch']}", trial_labels=trial_conditions)
    # 绘制raster图，这里返回fig是演示后续可以保存图形，输入trial_labels可以按condition绘制不同条件的raster图y轴，
    # 输入show_psth=False可以不绘制PSTH图
    cluster_fig,_ = analyzer.plot_raster(suptitle=f"Cluster {clu_id} Raster & PSTH, on channel {clu_info.at[clu_id, 'ch']}", trial_labels=trial_conditions, show_psth=True)
    # cluster_fig.savefig(FIG_SAVE_PATH / f"cluster_{clu_id}_raster_psth.png", dpi=300, bbox_inches='tight')
    # plt.close(cluster_fig)  # 关闭图形以释放内存

In [ ]:
# 可以直接访问analyzer的属性来获取对齐后的spike train
# 这是一个list，每个元素代表了一个trial中，这个cluster的发放时间（相对于对齐事件，秒为单位）
analyzer.aligned_spike_train

# 可以直接访问analyzer的rates属性来获取PSTH结果
# 这是一个2维数组，第一维度是trial，第二维是时间点
# 时间点精度由之前计算calculate_rates方法决定
analyzer.rates

In [ ]:
# 可以用SpikeTrainAnalyzer来分析多个cluster的spike train
# 这里假设你已经有了一个显著的cluster ID列表

from tqdm.notebook import tqdm
# 构建事件窗口数组,假设每个trial的事件窗口是以事件26的时间为中心，前后各5秒
event_start_end_array = np.column_stack((event_26_time_list-5, event_26_time_list+5))

cebra_st = []
for clu_id in tqdm(significant_clusters):
    spike_train = np.array(st[clu==clu_id]) / F_S
    analyzer = SpikeTrainAnalyzer(
        spike_train=spike_train, 
        event_windows=event_start_end_array, 
        alignment_times=event_26_time_list,
        extra_events=extra_events,
        font_prop=my_font
    )
    analyzer.calculate_rates(
        mode='gaussian', 
        gaussian_std=40e-3,
        high_res_bin=1e-3,
        analysis_window=[-5,3]
    )
    # rates for this cluster:
    cebra_st.append(analyzer.rates)
# st is a list of rates for each significant cluster
cebra_st = np.array(cebra_st)  # N_neuron, N_trial, N_time
# 对于每个trial，应该有自己的st
cebra_st = cebra_st.transpose(1, 2, 0)  # (N_trial, N_time, N_neuron)

# 到此为止,你就获得了一个三维数组，形状为(N_trial, N_time, N_neuron)，
# 你可以用它们来降维分析,也可以用于cebra的分析

# 将第一个维度变成list，也就是变成一个list，拥有N_trial个元素，每个元素是一个N_neuron x N_time的数组
# 可以用来进行cebra的 Multi-session 分析
cebra_st = [cebra_st[i] for i in range(cebra_st.shape[0])]
